<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/branch_leo/IMECOS_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# initialize the database, see import statements in *implementation.py
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/source_database_implementation.py').text)

In [ ]:
# decions about the design of the star scheme have followed several deeper looks into the data. Included here for documentation purpose.
# EXECUTION OF THIS CELL IS OPTIONAL!
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/branch_leo/data_clearing_SQL_queries.py').text)

In [ ]:
data_clearing_queries(30)

In [ ]:
# ETL process
# CREATE VIEWS
cur.execute("""
CREATE VIEW v_encounters_plus AS 
SELECT
  tb_encounters.Id,
  tb_encounters.DATASET_ORIGIN,
  tb_encounters.START as encounter_START,
  tb_encounters.STOP as encounter_STOP,
  tb_encounters.PATIENT_Id,
  tb_encounters.PAYER_Id,
  tb_payer_transitions.START_YEAR AS payer_START_YEAR,
  tb_payer_transitions.END_YEAR AS payer_END_YEAR,
  tb_payer_transitions.OWNERSHIP AS payer_OWNERSHIP,
  tb_encounters.ENCOUNTERCLASS,
  tb_encounters.CODE as encounter_CODE,
  tb_encounters.DESCRIPTION as encounter_DESCRIPTION,
  tb_encounters.REASONCODE as encounter_REASONCODE,
  tb_encounters.REASONDESCRIPTION as encounter_REASONDESCRIPTION,
  tb_conditions.START as condition_START,
  tb_conditions.STOP as condition_STOP,
  tb_conditions.CODE as condition_CODE,
  tb_conditions.DESCRIPTION as condition_DESCRIPTION,
  tb_immunizations.DATE as immunization_DATE,
  tb_immunizations.CODE as immuization_CODE,
  tb_immunizations.DESCRIPTION as immunizations_DECRIPTION,
  tb_procedures.DATE as procedure_DATE,
  tb_procedures.CODE as procedure_CODE,
  tb_procedures.DESCRIPTION as procedure_DESCRIPTION,
  tb_procedures.REASONCODE as procedure_REASONCODE,
  tb_procedures.REASONDESCRIPTION as procedure_REASONDESCRIPTION,
  tb_medications.START as medication_START,
  tb_medications.STOP as medication_STOP,
  tb_medications.CODE as medication_CODE,
  tb_medications.DESCRIPTION as medication_DESCRIPTION,
  tb_medications.REASONCODE as medication_REASONCODE,
  tb_medications.REASONDESCRIPTION as medication_REASONDESCRIPTION,
  tb_encounters.BASE_ENCOUNTER_COST,
  tb_immunizations.BASE_COST as BASE_IMMUNIZATION_COST,
  tb_procedures.BASE_COST as BASE_PROCEDURE_COST,
  tb_medications.BASE_COST as BASE_MEDICATION_COST,
  tb_medications.PAYER_COVERAGE as medication_PAYER_COVERAGE,
  tb_medications.DISPENSES as medication_DISPENSES,
  tb_medications.TOTALCOST as medications_TOTALCOST,
  CASE WHEN tb_medications.PAYER_ID = tb_encounters.PAYER ID THEN 'Versicherungsgleich' ELSE NULL END as medication_payer_class
FROM
  tb_encounters,
  tb_payer_transitions,
  tb_conditions,
  tb_immunizations,
  tb_procedures,
  tb_medications
WHERE
  tb_payer_transitions.PAYER_ID = tb_encounters.PAYER_Id      AND
  tb_payer_transitions.PATIENT_Id = tb_encounters.PATIENT_Id  AND
  tb_conditions.ENCOUNTER_Id = tb_encounters.Id               AND
  tb_conditions.ENCOUNTER_DSO = tb_encounter.DATASET_ORIGIN   AND
  tb_conditions.PATIENT_Id = tb_encounters.PATIENT_Id         AND
  tb_immunizations.ENCOUNTER_Id = tb_encounters.Id               AND
  tb_immunizations.ENCOUNTER_DSO = tb_encounter.DATASET_ORIGIN   AND
  tb_immunizations.PATIENT_Id = tb_encounters.PATIENT_Id         AND
  tb_procedures.ENCOUNTER_Id = tb_encounters.Id               AND
  tb_procedures.ENCOUNTER_DSO = tb_encounter.DATASET_ORIGIN   AND
  tb_procedures.PATIENT_Id = tb_encounters.PATIENT_Id         AND
  tb_medications.ENCOUNTER_Id = tb_encounters.Id               AND
  tb_medications.ENCOUNTER_DSO = tb_encounter.DATASET_ORIGIN   AND
  tb_medications.PATIENT_Id = tb_encounters.PATIENT_Id
""")
conn.commit()
cur.execute("""
SELECT
  *
FROM
  v_encounters_plus
WHERE
  medication_payer_class <> 'Versicherungsgleich'
"""
)
for row in (cur.fetchall()):
  print(row)
